In [1]:
%use lets-plot

In [2]:
data class SimStep(val sim:Int, 
                   val horizon:Int, 
                   val armNumber:Int, 
                   val reward:Float, 
                   val cumulativeReward:Float)


In [3]:
val steps = java.io.File("cli-bandit/target/")
    .listFiles()
    .filter { it.name.endsWith("tsv") }
    .map { Pair(
            it.name, 
            java.io.File(it.absolutePath).readLines()
                .map { it.split("\t") }
                .map { SimStep(it[0].toInt(), it[1].toInt(), it[2].toInt(), it[3].toFloat(), it[4].toFloat()) }
                .groupBy { it.horizon }
        )
    }

In [4]:
val metrics = steps.map { p -> Pair(
            p.first, 
            Triple(
                 p.second.keys.toList(), 
                 p.second.values.map{ sim -> sim.map{ it.reward }.average() },
                 p.second.values.map{ sim -> sim.map{ it.cumulativeReward }.average() }
            ) 
        )
    }

In [6]:
val horizon =  "horizon" to metrics.map { p1 -> p1.second.first }.sortedByDescending {it.size}.first()

val mapPlot = metrics.flatMap { listOf<Pair<String, Any?>>(
        "reward ${it.first}:" to it.second.second
//        "cumulative ${it.first}" to it.second.third.map { ti -> ti /  it.second.third.max()!! }
    )
}.toMap().plus(horizon)



In [12]:
val random = java.util.Random()
var plot = lets_plot(mapPlot)
mapPlot.keys.filter { it != "horizon" }.forEach {
        val color = 0x1000000 + random.nextInt(0x1000000);
        val colorString = "#" + Integer.toHexString(color).substring(1, 7)
        plot += geom_line(color = colorString){ x="horizon"; y = it ;alpha=0.1}
        val values = mapPlot[it] as List<Double>
        plot += geom_text(color = colorString, label = it.replace(".tsv", ""), x=150.0 , y = values[150])
}

plot + ggsize(1000, 900) 